In [1]:
!pip install -q transformers datasets tokenizers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Домашнее задание. Transformers.

Привет! Это очередное домашнее задание, но теперь ты познакомишься с моделя трансформеров и с библиотекой `HuggingFace🤗`. В этом задании будет предложено решить с помощью модели `GPT2` простую задачу (анализ сентимента) и сделать небольшое исследование карт внимания. Приступим!

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker

import torch
import torch.nn as nn

from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, GPT2Config
from datasets import load_dataset

In [4]:
device = "cuda" if torch.cuda.is_available else "cpu"
device = "cpu"
print(device)

cpu


Датасет, который мы будем использовать сегодня – тексты из английского твиттера. Они уже почищенны от никнеймов, знаков препинания и прочего. 

In [5]:
emotion_dataset = load_dataset("emotion")

Using custom data configuration default
Reusing dataset emotion (/root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

Посмотри, из чего состоит `emotion_dataset`:

In [6]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
emotion_dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [8]:
emotion_dataset["train"]["text"][0]

'i didnt feel humiliated'

In [9]:
len(np.unique(emotion_dataset["validation"]["label"]))

6

In [10]:
len(emotion_dataset["train"])

16000

Для перевода текста в токены мы будем использовать предобученный BPE-токенайзер.

In [11]:
tokenizer = GPT2TokenizerFast.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token # У gpt2 нет pad токенов. Вместо них воспользуемся токенами конца текста.

Подготовь класс, который принимает датасет, токенайзер и имя используемой части (`train`, `validation`, `test`). Используй его для получения данных для обучения.

P.S. Посмотри, как работает токенайзер ([docs](https://huggingface.co/transformers/main_classes/tokenizer.html)) и подумай, как его надо добавить в датасет.

Немного примеров, как работает токенайзер. Это поможет с написанием датасета.

In [12]:
tokenizer.tokenize(emotion_dataset["train"]["text"][0])

['i', 'Ġdidnt', 'Ġfeel', 'Ġhumiliated']

In [13]:
tokenizer.encode(emotion_dataset["train"]["text"][0])

[72, 42547, 1254, 42659]

In [14]:
tokenizer.encode_plus(emotion_dataset["train"]["text"][0])

{'input_ids': [72, 42547, 1254, 42659], 'attention_mask': [1, 1, 1, 1]}

In [15]:
tokenizer.encode_plus(emotion_dataset["train"]["text"][0], return_tensors="pt")

{'input_ids': tensor([[   72, 42547,  1254, 42659]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [16]:
tokenizer.encode_plus(
    emotion_dataset["train"]["text"][0], 
    max_length=128, # максимальная длина текста
    padding="max_length", # надо ли добавлять паддинг в конце?
    return_tensors="pt", # возвращает pytorch тензоры
)

{'input_ids': tensor([[   72, 42547,  1254, 42659, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 5

In [17]:
# Если надо, попрактикуйся работать с токенайзером здесь

In [18]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, part, dataset=emotion_dataset, tokenizer=tokenizer, max_length=128):
        """
        part: "train", "validation", "test"
        """
        self.part = part
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        self.labels = np.unique(dataset[part]["label"])
        self.label2num = {l: num for num, l in enumerate(self.labels)}
        
    def __getitem__(self, idx):
        """
        Return dict with tokens, attention_mask and label
        """
        text = self.dataset[self.part]["text"][idx]
        label = self.dataset[self.part]["label"][idx]
        
        tokenizer_output = tokenizer.encode_plus(
            text, 
            max_length=self.max_length, # максимальная длина текста
            padding="max_length", # надо ли добавлять паддинг в конце?
            return_tensors="pt", # возвращает pytorch тензоры
        )
        target = self.label2num[label]
        return {
            "input_ids": tokenizer_output["input_ids"][0], 
            "mask": tokenizer_output["attention_mask"][0],
            "target": target
        }
        
    def __len__(self):
        """
        Return length of dataset
        """
        return len(self.dataset[self.part])

Создай `train`, `validation` и `test` части датасета. Загрузи их в `DataLoaders`.

In [19]:
train_dataset = TweetDataset(part="train")
valid_dataset = TweetDataset(part="validation") # validation
test_dataset = TweetDataset(part="test")

In [20]:
batch_size = 64 # Задай batch_size

train_loader = torch.utils.data.DataLoader(
  train_dataset, batch_size, shuffle=True
)
valid_loader = torch.utils.data.DataLoader(
  train_dataset, batch_size, shuffle=False
)
test_loader = torch.utils.data.DataLoader(
  train_dataset, batch_size, shuffle=False
)

## Начнем с нуля.

Попробуем обучить модель трансформер с нуля решать данную задачу.

In [21]:
config = GPT2Config.from_pretrained(
    "distilgpt2", # distilgpt2 – уменьшенная версия модели gpt2
    output_attentions=True,
    pad_token_id=tokenizer.eos_token_id,
    num_labels=len(np.unique(emotion_dataset["train"]["label"]))
)
model_0 = GPT2ForSequenceClassification(config=config).to(device) # GPT2 для классификации текста

In [22]:
model_0

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid

Подготовь оптимайзер и критерий:

In [23]:
lr = 1e-5 # Предполагаемый learning rate. Он может быть больше или меньше :)

optimizer = torch.optim.Adam(model_0.parameters(), lr)
criterion = torch.nn.CrossEntropyLoss()
# scheduler = ... # Можно добавить шедулер для обучения моделей. Это на твое усмотрение

Посмотри, что возвращает модель ([docs](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2forsequenceclassification)), если в неё подать данные:

In [24]:
tokenized_sample = tokenizer.encode_plus(
    emotion_dataset["train"]["text"][0], 
    max_length=128, # максимальная длина текста
    padding="max_length", # надо ли добавлять паддинг в конце?
    return_tensors="pt", # возвращает pytorch тензоры
)

tokens = tokenized_sample["input_ids"].to(device) # Получи токены из датасета
mask = tokenized_sample["attention_mask"].to(device) # Получи маску из датасета

output = model_0(tokens, attention_mask=mask) # Посмотри на аутпут

In [25]:
tokens = torch.cat((tokenized_sample["input_ids"], tokenized_sample["input_ids"])).to(device) # Получи токены из датасета
mask = torch.cat((tokenized_sample["attention_mask"], tokenized_sample["attention_mask"])).to(device) # Получи маску из датасета

output = model_0(tokens, attention_mask=mask)

Обучи модель с помощью `train_dataset`, проверяй во время обучения качество с помощью `valid_dataset` и получи финальное качество с помощью `test_dataset`.

In [26]:
tokens = tokenized_sample["input_ids"].to(device) # Получи токены из датасета
mask = tokenized_sample["attention_mask"].to(device) # Получи маску из датасета

model_0(tokens, attention_mask=mask) # Посмотри на аутпут

SequenceClassifierOutputWithPast([('logits',
                                   tensor([[-0.1100,  0.3030,  0.1940,  0.2948, -0.3990, -0.5319]],
                                          grad_fn=<IndexBackward0>)),
                                  ('past_key_values',
                                   ((tensor([[[[-0.0582, -0.4053, -0.5964,  ...,  1.3937,  0.7288, -0.3292],
                                               [-0.2221, -0.7305,  0.3945,  ...,  0.1327,  0.2910, -0.7653],
                                               [ 0.4005,  0.4438, -0.1183,  ...,  0.7425,  0.6119, -0.1645],
                                               ...,
                                               [-0.2586,  0.7456, -0.2243,  ...,  0.2290, -0.0045,  0.4528],
                                               [ 0.2110,  0.3447,  0.4109,  ...,  0.2539,  0.1476,  0.5436],
                                               [-0.4976,  0.7130,  0.5152,  ..., -0.1710,  0.5580,  0.7337]],
                        

In [27]:
from tqdm.notebook import tqdm

num_epochs = 10
# Train loop
for e in range(num_epochs):
    model_0.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        output = model_0(batch["input_ids"].to(device), attention_mask=batch["mask"].to(device))
        loss = criterion(output["logits"], batch["target"].to(device))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        
    valid_loss = 0
    valid_acc = 0
    model_0.eval()
    with torch.no_grad():
        for batch in valid_loader:
            output = model_0(batch["input_ids"].to(device), attention_mask=batch["mask"].to(device))
            loss = criterion(output["logits"], batch["target"].to(device))
            valid_loss += loss.item()
            valid_acc += (batch["target"].to(device) == output["logits"].argmax(axis=1)).sum().item()
            

    print(f"Train Loss: {train_loss / (len(train_loader) * batch_size)}, "
          f"Valid Loss: {valid_loss / (len(valid_loader) * batch_size)}, "
          f"Valid Acc: {valid_acc / (len(valid_loader) * batch_size)}")
    
    torch.save(model.state_dict(), '/content/drive/gpt2_best_model.pt')
    
        
# Testing
test_acc = 0
model_0.eval()
with torch.no_grad():
    for batch in test_loader:
        output = model_0(batch["input_ids"].to(device), attention_mask=batch["mask"].to(device))
        test_acc += (batch["target"].to(device) == output["logits"].argmax(axis=1)).sum().item()
        
print(f"\n\nTest Acc: {test_acc / len(test_loader)}")

  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [97]:
! pip install pynput

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 8.2 MB/s 
     |████████████████████████████████| 179 kB 69.8 MB/s 
  Created wheel for evdev: filename=evdev-1.5.0-cp37-cp37m-linux_x86_64.whl size=97548 sha256=f45cbbf44bb24616387e26c1a9038cf9e8b6e20ddb636c93f1717acec03d7fb4
  Stored in directory: /root/.cache/pip/wheels/92/22/83/8cce4c52d67fca4bf2eb7a1de1ecd9053b992d9b95efae3fad
Successfully built evdev


После того, как получил лучшую модель для решения этой задачи, посмотри на карты внимания. Нашел ли что-нибудь интересное в них?

In [ ]:
def get_attention_matrixes(model, tokenizer, text, device=device):
    inp = list(filter(lambda x: x != tokenizer.sep_token_id, tokenizer.encode(text)))
    inp = torch.tensor(inp, dtype=torch.long, device=device).unsqueeze(0)
    attn_tensors = model(inp)[-1]
    seq = [tokenizer.decode(x) for x in inp[0].tolist()]
    attn = []
    for i in range(len(attn_tensors)):
        attn_layer = []
        for j in range(attn_tensors[i].size(1)):
            attn_layer.append(attn_tensors[i][0, j].cpu().detach().numpy())
        attn.append(np.array(attn_layer))
    
    return np.array(attn)

In [ ]:
def show_attention(seq, attentions):
    # Set up figure with colorbar
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions)
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels(['']+seq, rotation=90, fontsize=16)
    ax.set_yticklabels(['']+seq, fontsize=16)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
text = ... # Выбери текст из датасета
tokens = tokenizer.tokenize(text)

In [ ]:
attns = get_ettention_matrixes(model_0, tokenizer, text)
show_attention(seq, attn[-1][0])

## Fine-tuning

Теперь другой подход: загрузим модель, которая обучалась решать задачу Language Modeling. Посмотрим, получим ли мы прирост в качестве.

In [ ]:
model_1 = GPT2ForSequenceClassification.from_pretrained(
    "distilgpt2", 
    output_attentions=True,
    pad_token_id=tokenizer.eos_token_id,
    num_labels=8
).to(device)

In [ ]:
lr = 1e-5 # Предполагаемый learning rate. Он может быть больше или меньше :)

optimizer = ...
criterion = ...
# scheduler = ... # Можно добавить шедулер для обучения моделей. Это на твое усмотрение

Вывод модели ничем не отличается от предыдущего случая, поэтому сразу приступаем к обучению:

In [ ]:
from tqdm.notebook import tqdm


num_epochs = 10

# Train loop
for e in range(num_epochs):
    model_1.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        ...
        
    valid_loss = 0
    valid_acc = 0
    model_1.eval()
    with torch.no_grad():
        for batch in valid_loader:
            ...
    print(f"Train Loss: {train_loss / len(train_loader)},"
          f"Valid Loss: {valid_loss / len(valid_loader)},"
          f"Valid Acc: {valid_acc / len(valid_loader)}")
        
# Testing
test_acc = 0
model_1.eval()
with torch.no_grad():
    for batch in test_loader:
        ...
print(f"Test Acc: {test_acc / len(test_loader)}")

Есть ли прирост качества или скорости обучения?

Посмотри на карты внимания. Есть ли отличие от предыдущего случая?

In [ ]:
text = ... # Выбери текст из датасета
tokens = tokenizer.tokenize(text)

In [ ]:
attns = get_ettention_matrixes(model, tokenizer, text)
show_attention(seq, attn[-1][0])

## Отчет

Покажи здесь, что ты выполнил по этой работе. Ответь на несколько вопросов:
- Какой подход оказался лучше? 
- На какие слова модель большего всего обращала внимание?
- На каких слоях/головах модель обращала внимание?

< твой отчет/ответы >